# **Neural Style Transfer**

## Запуск обученной модели

In [ ]:
# Общий вид запуска программы тестирования:
# python3 test_on_image.py --image_path <path-to-image> --checkpoint_model <path-to-checkpoint> 

# Конкретный пример запуска тестирования:
# python3 test_on_image.py --image_path /Users/macbookpro/bird.jpg --checkpoint_model /Users/macbookpro/checkpoints/результаты/kalzado_10000.pth 

## Программа тестирования обученной модели

Загружаем необходимые для работы программы библиотеки. При запуске программы из терминала мы можем также просто импортировать из ранее созданной программы models.py класс TransformerNet для загрузки весов обученной модели. Также мы импортируем все функции из ранее созданной программы utils.py.

In [9]:
from this import s
from models import TransformerNet
from utils import *
import torch
from torch.autograd import Variable
import argparse
import os
from torchvision.utils import save_image
from PIL import Image

import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms
import torch
import numpy as np

Реализация класса TransformerNet, для которого требуются дополнительные классы ResidualBlock и ConvBlock:

In [6]:
# Transformer network
class TransformerNet(torch.nn.Module):
    def __init__(self):
        super(TransformerNet, self).__init__()
        self.model = nn.Sequential(
            ConvBlock(3, 32, kernel_size = 9, stride = 1),
            ConvBlock(32, 64, kernel_size = 3, stride = 2),
            ConvBlock(64, 128, kernel_size = 3, stride = 2),
            ResidualBlock(128),
            ResidualBlock(128),
            ResidualBlock(128),
            ResidualBlock(128),
            ResidualBlock(128),
            ConvBlock(128, 64, kernel_size = 3, upsample = True),
            ConvBlock(64, 32, kernel_size = 3, upsample = True),
            ConvBlock(32, 3, kernel_size = 9, stride = 1, normalize = False, relu = False),
        )

    def forward(self, x):
        return self.model(x)


# Residual block
class ResidualBlock(torch.nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            ConvBlock(channels, channels, kernel_size = 3, stride = 1, normalize = True, relu = True),
            ConvBlock(channels, channels, kernel_size = 3, stride = 1, normalize = True, relu = False),
        )

    def forward(self, x):
        return self.block(x) + x


# Convolutional block
class ConvBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride = 1, upsample = False, normalize = True, relu = True):
        super(ConvBlock, self).__init__()
        self.upsample = upsample
        self.block = nn.Sequential(
            nn.ReflectionPad2d(kernel_size // 2), nn.Conv2d(in_channels, out_channels, kernel_size, stride)
        )
        self.norm = nn.InstanceNorm2d(out_channels, affine=True) if normalize else None
        self.relu = relu

    def forward(self, x):
        if self.upsample:
            x = F.interpolate(x, scale_factor = 2)
        x = self.block(x)
        if self.norm is not None:
            x = self.norm(x)
        if self.relu:
            x = F.relu(x)
        return x

Требуемые функции из программы utils.py:

In [ ]:
# Среднее значение и стандартное отклонение, используемые для предварительно обученных моделей PyTorch
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

def style_transform(image_size = None):
    """ Преобразование стилевых изображений """
    resize = [transforms.Resize(image_size)] if image_size else []
    transform = transforms.Compose(resize + [transforms.ToTensor(), transforms.Normalize(mean, std)])
    return transform

def denormalize(tensors):
    """ Денормализация тензоров изображений с использованием среднего и стандартного отклонения """
    for c in range(3):
        tensors[:, c].mul_(std[c]).add_(mean[c])
    return tensors

В начале мы проверяем верность введённых ключей, при их неправильном вводе выводим пользователю подсказку, создаём нужные директории при их изначальном отсутствии. После определяем модель и загружаем обученные веса по пути, заданному ключом "--checkpoint_model". Далее загружаем контентное изображение по пути, указанному ключом "--image_path", проводим стилизацию этого изображения, сохраняя сгенерированное изображение на локальный компьютер.

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--image_path", type = str, required = True, help = "Path to image")
    parser.add_argument("--checkpoint_model", type = str, required = True, help = "Path to checkpoint model")
    args = parser.parse_args()
    print(args)

    os.makedirs("images/outputs", exist_ok = True)

    device = torch.device("cpu")

    transform = style_transform()

    # Определение модели, загрузка весов модели 
    transformer = TransformerNet().to(device)
    transformer.load_state_dict(torch.load(args.checkpoint_model, map_location = 'cpu'))
    transformer.eval()

    # Загрузка контентного изображения
    image_tensor = Variable(transform(Image.open(args.image_path))).to(device)
    image_tensor = image_tensor.unsqueeze(0)

    # Стилизация изображения
    with torch.no_grad():
        stylized_image = denormalize(transformer(image_tensor)).cpu()

    # Сохранение сгенерированного изображения
    fn = args.image_path.split("/")[-1]
    save_image(stylized_image, f"stylized-{fn}")